In [ ]:
# 1. get FIPS from tigerweb
# 2. use FIPS to call Census stats API
# 3. merge data with GeoJSON

In [ ]:
# Step 1 get FIPS code
# Step 2 , use FIPS code to get geo data

In [ ]:
# save fig dpi = 

In [1]:
import pandas as pd
import geopandas as gpd
from geopandas.tools import sjoin
import folium
from folium import IFrame
from folium.plugins import MarkerCluster
# MarkerCluster()

In [2]:
shapefile = gpd.read_file("tl_2010_25025_tract10.shp")
print(shapefile)

    STATEFP10 COUNTYFP10 TRACTCE10      GEOID10   NAME10  \
0          25        025    010405  25025010405   104.05   
1          25        025    010404  25025010404   104.04   
2          25        025    010801  25025010801   108.01   
3          25        025    010702  25025010702   107.02   
4          25        025    010204  25025010204   102.04   
5          25        025    010802  25025010802   108.02   
6          25        025    010104  25025010104   101.04   
7          25        025    000703  25025000703     7.03   
8          25        025    000504  25025000504     5.04   
9          25        025    000704  25025000704     7.04   
10         25        025    010103  25025010103   101.03   
11         25        025    000803  25025000803     8.03   
12         25        025    980300  25025980300     9803   
13         25        025    120201  25025120201  1202.01   
14         25        025    120104  25025120104  1201.04   
15         25        025    110607  2502

In [3]:
df_geo = gpd.GeoDataFrame(shapefile)

In [4]:
df_geo['id']=df_geo['STATEFP10']+df_geo['COUNTYFP10']+df_geo['TRACTCE10']

In [5]:
df_geo = df_geo[['geometry','id']]

In [6]:
df_data = pd.read_csv('fullcount_csv',index_col=0)

In [7]:
df_pop = df_data[['tractid','POP00_POC']]

In [8]:
df_pop['id']=df_pop['tractid'].astype('str')

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df_pop.drop(columns=['tractid'],axis=1,inplace=True)

/Users/Kristinabarounis/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [10]:
df_tovisualize = df_geo.merge(df_pop, on='id')

In [21]:
# create basemap for Boston area
mapobj = folium.Map(location=[42.3601, -71.0589],zoom_start=13,tiles='Stamen Terrain')

In [12]:
bins=list(df_tovisualize['POP00_POC'].quantile([0,0.25,0.5,0.75,1]))

In [ ]:
bins

In [22]:
folium.features.Choropleth(geo_data = df_tovisualize.to_json(),
                  data=df_tovisualize,
                  columns=['id','POP00_POC'],
                           key_on='feature.properties.id',fill_color='BuGn',bins=bins,
                           legend_name='Change in population between 2000 and 2010',highlight=True).add_to(mapobj)

In [23]:
folium.LayerControl().add_to(mapobj)

In [24]:
def embed_map(m):
    from IPython.display import IFrame

    m.save('index.html')
    return IFrame('index.html', width='100%', height='750px')

In [25]:
embed_map(mapobj)

In [ ]:
df_tovisualize.head()